#### Outline
1. 读入KML文件，存为点云
2. 根据距离建立初始类别
3. 根据方向，距离等聚类获得每一条路的归属车点集(对于公交车上下行信息可以用于简单判断)
4. 筛选剔除不合适的点，如0速度点
5. 分时间计算平均速度，并写入KML文件

In [4]:
import numpy as np
import pandas as pd
import json 
import re

#### Section 1

In [5]:
from sklearn.neighbors import KDTree
import os

In [24]:
FilePath='E:/WorkSpace/py/travel_flow/data/roadnet/excel_ver'

PointSet=[]
Cnt=0

for root, dirs, files in os.walk(FilePath):
    for file in files:
        df=pd.read_excel(os.path.join(root, file),encoding='utf-8')
        df=pd.DataFrame(df['ns1:coordinates'])
        df.columns=['xy']

        for i in range(0,len(df)):
            if not isinstance(df['xy'][i], float):
                strs=df['xy'][i].split(' ')
                Cnt=Cnt+1
                for pairs in strs:
                    coordinates=pairs.split(',')
                    PointSet.append({'road_id':Cnt, 'co_x':float(coordinates[0]), 'co_y':float(coordinates[1])})
PointSet=pd.DataFrame(PointSet)
PointCloud=np.array(PointSet.loc[:,['co_x','co_y']])
PointSet
tree = KDTree(PointCloud)

In [10]:
print(PointCloud.shape)

(66614, 2)


#### Section 2

In [35]:
FilePath='E:/WorkSpace/py/travel_flow/data/sample_data_week/bus/'
FileName='bus-20200309.csv'
df=pd.read_csv(FilePath+FileName,encoding='utf-8')
df.drop(['???"DEVICEINDEXCODE"','LINEINDEXCODE','DIRECTION','CURRENTSTATIONINDEXCODE'],axis=1,inplace=True)
df.head()

,LINEDIR,LONGITUDE,LATITUDE,GPSTIME,SPEED
0,下行,120.594386,28.021144,2020/3/9 0:00,0.0
1,上行,120.648602,28.011177,2020/3/9 0:00,35.0
2,下行,120.717183,28.008483,2020/3/9 0:00,29.6
3,上行,120.599269,28.016125,2020/3/9 0:00,0.0
4,下行,120.838413,27.906455,2020/3/9 0:00,0.0


In [38]:
Road_dic={}
for i in range(0,len(df)):
    #0的直接剔除了_______-还没考虑路口，红绿灯等
    if not df['SPEED'][i]:
        continue
    Velxy=np.array(df.iloc[[i],[1,2]])
    dist,raw_ind = tree.query(Velxy,k=1)
    index=PointSet.loc[int(raw_ind)]['road_id'] 
#     if(dist<threshold):
    VelInfo={'Dir':df['LINEDIR'][i],'Lon':df['LONGITUDE'][i],'Lat':df['LATITUDE'][i],\
                               'GPS':pd.Timestamp(df['GPSTIME'][i]),'Speed':df['SPEED'][i]}
    if not index in Road_dic:
        Road_dic[index]=pd.DataFrame([VelInfo])
    else:
        Road_dic[index]=Road_dic[index].append(VelInfo,ignore_index=True)

#### Section 3

In [39]:
# 根据行进方向，剔除错误的点______尚未附加到正确的
for key in Road_dic.keys():
    df_=Road_dic[key]
    direction = df_['Dir'].value_counts().index[0]
    df_=df_[df_.isin([direction])]

#### Section 4

In [134]:
import dict2xml
from xml.dom.minidom import parseString

In [135]:
xml_str=''
RoadGroup=PointSet.groupby('road_id')

def wrap_mark(index,df):
    road_str=''
    groups=df.groupby(pd.Grouper(key='GPS',freq='T'))
    
    for time,group in groups:
        if (not group.empty):
            dic={}
            dic['name']=''
            dic['Style']={'LineStyle':{'color':'ffFFFF00'},'PolyStyle':{'fill':0}}
            Poly=RoadGroup.get_group(index)
            poly_str=''
            for idx in Poly.index:
                poly_str=poly_str+str(Poly['co_x'][idx])+','+str(Poly['co_y'][idx])+' '
            dic['LineString']={'coordinates':poly_str}
            dic['TimeStamp']={'when':str(time)}
            dic['description']=str(group['Speed'].mean())
            road_str=road_str+dict2xml.dict2xml(dic,wrap='Placemark',indent=" ")+'\n'
    return road_str

for key in Road_dic.keys():
    xml_str+=wrap_mark(key,Road_dic[key])

xml_raw = '<?xml version="1.0" encoding="utf-8" ?>\n<kml xmlns="http://www.opengis.net/kml/2.2">\n<Document id="root_doc">\n<Folder><name>roadnet</name>\n' +xml_str+'\n</Folder>\n</Document></kml>'

In [200]:
# 写入KML文件
filename = "test.kml"
f = open(filename, "w")
f.write(xml_raw)   
f.close()

#### Extra Section

In [ ]:
# 根据已有的道路标签匹配
RoadRef=pd.read_csv('data/RoadName.csv')

#建立点云搜索
PointCloud=np.array(RoadRef.loc[:,['polylon','polylat']])
tree = KDTree(PointCloud)

In [201]:
Result,RoadName=[],[]
groups=PointSet.groupby('road_id')
for id,road in groups:
    road=road.reset_index(drop=True)
    #每条待标记的路选取至多十个点
    size = min (10,len(road))
    poly=road.loc[:size,['co_x','co_y']]
    _,ind = tree.query(np.array(poly).reshape(-1,2),k=1) 
    #选择命中最多的路的名字作为标记
    RoadName.append(pd.DataFrame(RoadRef.iloc[[int(x) for  x in ind]]['name'])['name'].value_counts().index[0])
RoadName

['岭北路',
 '下河路',
 'G1513温丽高速',
 'G1513温丽高速',
 '虹三线',
 '虹三线',
 '高桐线',
 '桐后线',
 'G1513温丽高速',
 'G1513温丽高速',
 '岭北路',
 '桐后线',
 '高桐线',
 '正岙巷四十弄',
 '正岙巷四十弄',
 '虹三线',
 'S10温州绕城高速',
 '丁岙东路',
 'S10温州绕城高速',
 '康隆南路',
 '小山线',
 '虹三线',
 'G1513温丽高速',
 'G1513温丽高速',
 '正岙巷四十弄',
 'G1513温丽高速',
 'G1513温丽高速',
 '白塔路',
 'G15沈海高速',
 'G1513温丽高速',
 'G1513温丽高速',
 '文武路',
 'G15沈海高速',
 '丁岙西路',
 '白塔路',
 '小山线',
 '丁岙西路',
 '高桐线',
 'S10温州绕城高速',
 'S10温州绕城高速',
 'S10温州绕城高速',
 'S10温州绕城高速',
 'G15沈海高速',
 '瑞枫线',
 'S10温州绕城高速104国道出口',
 '浦东东路',
 '浦东东路',
 'G15沈海高速',
 'S10温州绕城高速104国道入口',
 '瑞枫线',
 '浦东东路',
 '瑞枫线',
 '浦东东路',
 'G1513温丽高速',
 '西象路',
 '西象路',
 '瑞枫线',
 '西象路',
 '西象路',
 '西象路',
 '江滨西路',
 '东浦巷',
 '西象路',
 '西象路',
 'G15沈海高速',
 'S10温州绕城高速104国道入口',
 'S10温州绕城高速104国道出口',
 'G15沈海高速',
 'G15沈海高速',
 'G1513温丽高速',
 'G15沈海高速',
 'G1513温丽高速',
 'S10温州绕城高速',
 'S10温州绕城高速',
 'G1513温丽高速',
 '瑞枫线',
 'G1513温丽高速',
 'G1513温丽高速',
 '瑞枫线',
 'S10温州绕城高速104国道出口',
 '街路街',
 'G1513温丽高速',
 'G1513温丽高速',
 'S10温州绕城高速104国道入口',
 'G1513温丽高速',
 'G1513温丽高速',
 '富阳南路',
 '富阳南路'

In [189]:
Road_Info=[]
groups=PointSet.groupby('road_id')
for id,g in groups:
    poly=''
    for coordinate in zip(g['co_x'],g['co_y']):
        poly+=str(coordinate[0])+','+str(coordinate[1])+' '
    poly+=';'
    Road_Info.append({'name':RoadName[int(id)-1],'Poly':poly})
Road_Info=pd.DataFrame(Road_Info)
Road_Info

,name,Poly
0,岭北路,"120.5495088,27.9033452 120.5502851,27.9052568 ..."
1,下河路,"120.56352330000001,28.0444443 120.5614825,28.0..."
2,G1513温丽高速,"120.6387915,27.9504138 120.6301696,27.9526361 ;"
3,G1513温丽高速,"120.6306033,27.9521021 120.6332219,27.9514481 ..."
4,虹三线,"120.70723230000002,28.0744747 120.7071197,28.0..."
...,...,...
7999,垟儿路,"120.7657577,28.0055182 120.7655119,28.0053117 ;"
8000,中心路,"120.7724617,27.9985548 120.7726809,27.9985013 ;"
8001,飞霞南路,"120.662187,28.0034417 120.6622024,28.0032354 ;"
8002,中心路,"120.7713349,27.9974934 120.7706921,27.9964628 ;"


In [199]:
#写入文件
NewRoad_Info=[]
groups = Road_Info.groupby('name')
for label,g in groups:
        Polys=''
        cnt=0
        for poly in g['Poly']:
            Polys+=poly
            cnt+=1
            if(cnt>150):
                cnt=0
                NewRoad_Info.append({'name':label,'Polyes':Polys})
                Polys=''
        NewRoad_Info.append({'name':label,'Polyes':Polys})
pd.DataFrame(NewRoad_Info).to_csv('data/Road_info.csv',encoding='utf_8_sig',index=0,header=0)

In [185]:
Result=[]
for key in Road_dic.keys():
         groups=Road_dic[key].groupby(pd.Grouper(key='GPS',freq='T'))
         for t,g in groups:
                 if (not g.empty):  
                          Result.append({'name':RoadName[int(key)-1],'Time':str(t),'Speed':str(g['Speed'].mean())})
Result

[{'name': '人民东路', 'Time': '2020-03-09 00:00:00', 'Speed': '35.0'},
 {'name': '人民东路', 'Time': '2020-03-09 06:03:00', 'Speed': '31.6'},
 {'name': '人民东路', 'Time': '2020-03-09 06:06:00', 'Speed': '34.275'},
 {'name': '人民东路',
  'Time': '2020-03-09 06:07:00',
  'Speed': '26.066666666666666'},
 {'name': '人民东路', 'Time': '2020-03-09 06:08:00', 'Speed': '20.0'},
 {'name': '人民东路', 'Time': '2020-03-09 06:18:00', 'Speed': '13.6'},
 {'name': '人民东路', 'Time': '2020-03-09 06:19:00', 'Speed': '17.5'},
 {'name': '人民东路',
  'Time': '2020-03-09 06:20:00',
  'Speed': '34.800000000000004'},
 {'name': '人民东路', 'Time': '2020-03-09 06:21:00', 'Speed': '21.55'},
 {'name': '人民东路',
  'Time': '2020-03-09 06:22:00',
  'Speed': '20.766666666666666'},
 {'name': '人民东路', 'Time': '2020-03-09 06:28:00', 'Speed': '33.2'},
 {'name': '人民东路', 'Time': '2020-03-09 06:29:00', 'Speed': '45.9'},
 {'name': '人民东路', 'Time': '2020-03-09 06:30:00', 'Speed': '31.45'},
 {'name': '人民东路',
  'Time': '2020-03-09 06:31:00',
  'Speed': '23.76666

In [186]:
pd.DataFrame(Result).to_csv('data/NewSample.csv',encoding='utf_8_sig',index=0)